In [ ]:
!!git clone https://github.com/zykls/folktables.git


["Cloning into 'folktables'...",
 'remote: Enumerating objects: 423, done.\x1b[K',
 'remote: Counting objects:   0% (1/173)\x1b[K',
 'remote: Counting objects:   1% (2/173)\x1b[K',
 'remote: Counting objects:   2% (4/173)\x1b[K',
 'remote: Counting objects:   3% (6/173)\x1b[K',
 'remote: Counting objects:   4% (7/173)\x1b[K',
 'remote: Counting objects:   5% (9/173)\x1b[K',
 'remote: Counting objects:   6% (11/173)\x1b[K',
 'remote: Counting objects:   7% (13/173)\x1b[K',
 'remote: Counting objects:   8% (14/173)\x1b[K',
 'remote: Counting objects:   9% (16/173)\x1b[K',
 'remote: Counting objects:  10% (18/173)\x1b[K',
 'remote: Counting objects:  11% (20/173)\x1b[K',
 'remote: Counting objects:  12% (21/173)\x1b[K',
 'remote: Counting objects:  13% (23/173)\x1b[K',
 'remote: Counting objects:  14% (25/173)\x1b[K',
 'remote: Counting objects:  15% (26/173)\x1b[K',
 'remote: Counting objects:  16% (28/173)\x1b[K',
 'remote: Counting objects:  17% (30/173)\x1b[K',
 'remote: Counting obje

In [1]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import xgboost as xgb


In [2]:
df = pd.read_csv("sampled_data_7000.csv")
df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,gender,native-country,income,occupation
0,26,17,0,0,Private,9th,5,Never-married,Own-child,Black,Male,United-States,2515,Other-service
1,40,47,0,1977,Self-emp-not-inc,Some-college,10,Married-civ-spouse,Husband,White,Male,United-States,99999,Craft-repair
2,42,48,0,0,Private,Assoc-acdm,12,Divorced,Unmarried,White,Female,United-States,18045,Exec-managerial
3,40,25,0,0,State-gov,Some-college,10,Married-civ-spouse,Husband,White,Male,United-States,22412,Protective-serv
4,40,45,0,0,State-gov,Some-college,10,Never-married,Not-in-family,White,Female,United-States,30100,Adm-clerical


In [3]:
# Drop education-num column (we hot-encode it with education column)
df = df.drop(columns=['education-num'])

#  List categorical columns to encode
categorical_cols = ['workclass', 'education', 'marital-status', 'relationship',
                    'race', 'gender', 'native-country', 'occupation']

#  One-hot encode categorical columns
df_train_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

## READY TO TRAIN!!

X = df_train_encoded.drop(columns=['income'])
y = df_train_encoded['income']

# Bin continuous target into quartiles for stratification
y_binned = pd.qcut(y, q=4, labels=False)

# Stratified train/val split using binned labels (to balance the income label in train&split)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y_binned,
    random_state=42
)


# Get SP

In [44]:
# prot_attrs = [col for col in X_train.columns if col.startswith('race_') or col.startswith('gender_')]

# y_train_float = np.array(y_train, dtype=np.float64)

# ph = phi.phi_control(pd.Series(y_train_float))
# phis = np.array(phi.phi(pd.Series(y_train_float), ph), dtype=np.float64)
# loss = id_loss(ph, X_train, prot_attrs, approx=False)

# model = xgb.XGBRegressor(
#     objective=loss,
#     n_estimators=30,
#     max_depth=3,
#     learning_rate=0.2,
#     random_state=42,
#     n_jobs=-1
# )


xgb_reg = xgb.XGBRegressor(
    # objective=loss,
    n_estimators=30,
    max_depth=3,
    learning_rate=0.2,
    random_state=42,
    n_jobs=-1
)


xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [45]:
pred_df = X_val.copy()
pred_df['pred_income'] = y_pred
pred_df['pred_income'] = pred_df['pred_income'].round(2)
pred_df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,pred_income
4109,40,51,0,1887,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,59217.031250
5295,40,26,0,0,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,32876.101562
3334,40,32,0,0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,39836.429688
6151,12,17,0,0,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,3569.870117
2081,35,23,0,0,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,14565.809570


In [46]:
mean_race_pred = {}
mean_gender_pred = {}


mean_race_pred['Privileged Race']=pred_df[pred_df['race_White']==True]['pred_income'].mean()
mean_race_pred['Not Privileged Race']=pred_df[pred_df['race_White']!=True]['pred_income'].mean()


mean_gender_pred['gender_Male']=pred_df[pred_df['gender_Male']==True]['pred_income'].mean()
mean_gender_pred['gender_Female']=pred_df[pred_df['gender_Male']!=True]['pred_income'].mean()

mean_race_pred

{'Privileged Race': np.float32(34992.29),
 'Not Privileged Race': np.float32(27005.775)}

In [47]:
race_sp = pd.DataFrame(0.0, index=mean_race_pred.keys(), columns=mean_race_pred.keys())

for i in race_sp.index:
  for j in race_sp.columns:
    race_sp.loc[i, j] = mean_race_pred[i] - mean_race_pred[j]

race_sp

,Privileged Race,Not Privileged Race
Privileged Race,0.000000,7986.513672
Not Privileged Race,-7986.513672,0.000000


In [48]:
gender_sp = pd.DataFrame(0.0, index=mean_gender_pred.keys(), columns=mean_gender_pred.keys())

for i in gender_sp.index:
  for j in gender_sp.columns:
    gender_sp.loc[i, j] = mean_gender_pred[i] - mean_gender_pred[j]

gender_sp

,gender_Male,gender_Female
gender_Male,0.000000,16012.115234
gender_Female,-16012.115234,0.000000
